# 🏭 실습 1: 제조 데이터 맵 & 비전 검사 입문

**소요시간: 2시간 | 난이도: ★☆☆☆☆ (입문)**

---

## 🎯 학습 목표

- 제조 현장의 OT/IT 구조(PLC, SCADA, MES)를 이해합니다
- 제조 데이터 유형(이미지, 시계열, 텍스트)을 구분하고 활용합니다
- 전이학습 기반 OK/NG 분류 모델을 직접 구현합니다
- **YOLOv8을 활용한 결함 탐지(Object Detection)를 실습합니다**

---

## 📋 사전 준비

- Google Colab 계정 (무료 GPU 사용)
- 기본 Python 문법 이해
- 딥러닝 기초 개념 (CNN이 무엇인지 정도)

---

# Part 1: 제조 현장 OT/IT 구조 이해 (30분)

## 1.1 제조 시스템 계층 구조

제조 현장은 크게 세 가지 계층으로 구성됩니다:

| 계층 | 시스템 | 역할 |
|------|--------|------|
| **현장 계층** | PLC (Programmable Logic Controller) | 센서/액추에이터 직접 제어 |
| **감시 계층** | SCADA (Supervisory Control) | 실시간 모니터링 및 제어 |
| **관리 계층** | MES (Manufacturing Execution) | 생산 계획, 품질 관리, 추적 |

## 1.2 제조 데이터 유형

| 데이터 유형 | 예시 | AI 활용 |
|-------------|------|--------|
| **이미지** | 카메라 촬영 제품 사진 | 비전 검사, 결함 탐지 |
| **시계열** | 온도, 압력, 진동 센서 데이터 | 예지보전, 이상 탐지 |
| **텍스트/테이블** | 작업일지, 품질 기록 | 공정 최적화, 수율 예측 |

## 1.3 비전 검사 방식 비교

| 방식 | 모델 예시 | 출력 | 적합한 상황 |
|------|-----------|------|-------------|
| **분류 (Classification)** | ResNet, EfficientNet | OK/NG 라벨 | 전체 제품의 양/불 판정 |
| **객체 탐지 (Object Detection)** | YOLO, Faster R-CNN | 바운딩 박스 + 클래스 | 결함 위치 및 종류 특정 |
| **세그멘테이션 (Segmentation)** | U-Net, Mask R-CNN | 픽셀 단위 마스크 | 정밀한 결함 영역 측정 |

---

# Part 2: 전이학습 기반 OK/NG 분류 실습 (40분)

## 2.1 환경 설정

아래 코드를 순서대로 실행하세요.

###한글 깨짐 방지

In [ ]:
!apt-get install -y fonts-nanum*
!rm -rf /root/.cache/matplotlib/* # 폰트 캐시 재설정
# 런타임 다시 시작

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
#
path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = mpl.font_manager.FontProperties(fname=path).get_name()
plt.rcParams['font.family'] = font_name

In [ ]:
# Step 1: 필요한 라이브러리 설치 및 임포트
!pip install -q torch torchvision

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import random
import os

print(f'PyTorch 버전: {torch.__version__}')
print(f'GPU 사용 가능: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU 이름: {torch.cuda.get_device_name(0)}')

## 2.2 샘플 데이터 준비

실제 산업 환경에서는 MVTec AD 같은 데이터셋을 사용합니다.  
여기서는 시뮬레이션 데이터를 생성하여 실습합니다.

In [ ]:
# Step 2: 샘플 데이터 다운로드 및 구성
!mkdir -p data/train/OK data/train/NG data/val/OK data/val/NG

def create_sample_images():
    '''OK: 균일한 표면, NG: 스크래치가 있는 표면'''
    for split in ['train', 'val']:
        n = 50 if split == 'train' else 10
        for i in range(n):
            # OK 이미지: 깨끗한 회색 배경
            ok_img = np.ones((224, 224, 3), dtype=np.uint8) * 180
            ok_img = ok_img + np.random.randint(-10, 10, ok_img.shape).astype(np.uint8)
            Image.fromarray(ok_img).save(f'data/{split}/OK/{i}.png')

            # NG 이미지: 스크래치 추가
            ng_img = ok_img.copy()
            for _ in range(random.randint(1, 3)):
                x1, y1 = random.randint(0, 200), random.randint(0, 200)
                x2, y2 = x1 + random.randint(10, 50), y1 + random.randint(2, 5)
                ng_img[y1:y2, x1:x2] = [50, 50, 50]
            Image.fromarray(ng_img).save(f'data/{split}/NG/{i}.png')

create_sample_images()
print('✅ 샘플 데이터 생성 완료!')
print(f'   - 학습 데이터: OK {len(os.listdir("data/train/OK"))}개, NG {len(os.listdir("data/train/NG"))}개')
print(f'   - 검증 데이터: OK {len(os.listdir("data/val/OK"))}개, NG {len(os.listdir("data/val/NG"))}개')

In [ ]:
# 생성된 이미지 샘플 확인
fig, axes = plt.subplots(2, 4, figsize=(12, 6))

for i, ax in enumerate(axes[0]):
    img = Image.open(f'data/train/OK/{i}.png')
    ax.imshow(img)
    ax.set_title(f'OK Sample {i+1}')
    ax.axis('off')

for i, ax in enumerate(axes[1]):
    img = Image.open(f'data/train/NG/{i}.png')
    ax.imshow(img)
    ax.set_title(f'NG Sample {i+1}')
    ax.axis('off')

plt.suptitle('OK vs NG 샘플 이미지', fontsize=14)
plt.tight_layout()
plt.show()

## 2.3 전이학습 모델 구현

ResNet18을 기반으로 2-class 분류기를 만듭니다.

### 전이학습(Transfer Learning)이란?
- 대규모 데이터셋(ImageNet)으로 사전 학습된 모델을 가져와서
- 우리 문제(OK/NG 분류)에 맞게 미세 조정(Fine-tuning)하는 기법
- 적은 데이터로도 높은 성능을 달성할 수 있습니다

In [ ]:
# Step 3: 데이터 로더 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dataset = ImageFolder('data/train', transform=transform)
val_dataset = ImageFolder('data/val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

print(f'✅ 데이터 로더 설정 완료!')
print(f'   - 클래스: {train_dataset.classes}')
print(f'   - 클래스 인덱스: {train_dataset.class_to_idx}')
print(f'   - 학습 데이터: {len(train_dataset)}개')
print(f'   - 검증 데이터: {len(val_dataset)}개')

In [ ]:
# Step 4: 전이학습 모델 정의
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'사용 디바이스: {device}')

# 사전학습된 ResNet18 로드
model = models.resnet18(weights='IMAGENET1K_V1')

# 마지막 FC 레이어를 2-class로 변경 (OK/NG)
num_features = model.fc.in_features
print(f'원래 FC 입력 특성 수: {num_features}')
print(f'원래 FC 출력 클래스 수: 1000 (ImageNet)')

model.fc = nn.Linear(num_features, 2)  # 2개 클래스로 변경
print(f'변경된 FC 출력 클래스 수: 2 (OK/NG)')

model = model.to(device)

# 손실 함수와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print('\n✅ 전이학습 모델 준비 완료!')

## 2.4 모델 학습

In [ ]:
# Step 5: 학습 루프
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss, correct = 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()
    return total_loss / len(loader), correct / len(loader.dataset)

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    correct = 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        correct += (outputs.argmax(1) == labels).sum().item()
    return correct / len(loader.dataset)

# 학습 기록 저장
history = {'loss': [], 'train_acc': [], 'val_acc': []}

# 학습 실행
print('🚀 학습 시작!\n')
for epoch in range(5):
    loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
    val_acc = evaluate(model, val_loader)

    history['loss'].append(loss)
    history['train_acc'].append(train_acc)
    history['val_acc'].append(val_acc)

    print(f'Epoch {epoch+1}/5: Loss={loss:.4f}, Train Acc={train_acc:.2%}, Val Acc={val_acc:.2%}')

print('\n✅ 학습 완료!')

In [ ]:
# 학습 과정 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss 그래프
axes[0].plot(history['loss'], 'b-o', label='Training Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(True)

# Accuracy 그래프
axes[1].plot(history['train_acc'], 'b-o', label='Train Accuracy')
axes[1].plot(history['val_acc'], 'r-o', label='Validation Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training & Validation Accuracy')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

---

# Part 3: 혼동행렬 및 임계값 조정 실습 (20분)

## 3.1 혼동행렬(Confusion Matrix) 시각화

### 혼동행렬 구성요소
- **TP (True Positive)**: NG를 NG로 올바르게 예측
- **TN (True Negative)**: OK를 OK로 올바르게 예측
- **FP (False Positive)**: OK를 NG로 잘못 예측 (과검출)
- **FN (False Negative)**: NG를 OK로 잘못 예측 (과검출 누락) ⚠️ 가장 위험!

In [ ]:
# Step 6: 혼동행렬 생성
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

@torch.no_grad()
def get_predictions(model, loader):
    model.eval()
    all_preds, all_labels, all_probs = [], [], []
    for images, labels in loader:
        outputs = model(images.to(device))
        probs = torch.softmax(outputs, dim=1)
        all_probs.extend(probs[:, 1].cpu().numpy())  # NG 확률
        all_preds.extend(outputs.argmax(1).cpu().numpy())
        all_labels.extend(labels.numpy())
    return np.array(all_preds), np.array(all_labels), np.array(all_probs)

preds, labels, probs = get_predictions(model, val_loader)

# 혼동행렬 시각화
cm = confusion_matrix(labels, preds)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['OK (예측)', 'NG (예측)'],
            yticklabels=['OK (실제)', 'NG (실제)'],
            annot_kws={'size': 16})
plt.xlabel('예측', fontsize=12)
plt.ylabel('실제', fontsize=12)
plt.title('OK/NG 분류 혼동행렬', fontsize=14)
plt.show()

print('\n📊 분류 리포트:')
print(classification_report(labels, preds, target_names=['OK', 'NG']))

## 3.2 임계값 조정의 중요성

제조 현장에서는 **NG를 OK로 판정하는 것(FN, 과검출 누락)**이 치명적입니다.  
불량품이 고객에게 전달되면 큰 손실이 발생할 수 있기 때문입니다.

임계값을 조정하여 NG 검출율을 높일 수 있습니다:
- **임계값 낮춤** → NG 검출율 ↑, 오검출율 ↑
- **임계값 높임** → NG 검출율 ↓, 오검출율 ↓

In [ ]:
# Step 7: 임계값 조정
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]

print('📈 임계값별 성능 비교:\n')
print(f'{"임계값":^8} | {"NG 검출율(Recall)":^18} | {"오검출율(FPR)":^15} | {"정확도":^10}')
print('-' * 60)

results = []
for thresh in thresholds:
    adjusted_preds = (probs >= thresh).astype(int)
    cm = confusion_matrix(labels, adjusted_preds)
    tn, fp, fn, tp = cm.ravel()

    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)

    results.append({'threshold': thresh, 'recall': recall, 'fpr': fpr, 'accuracy': accuracy})
    print(f'{thresh:^8.1f} | {recall:^18.2%} | {fpr:^15.2%} | {accuracy:^10.2%}')

print('\n💡 제조 현장에서는 일반적으로 NG 검출율(Recall)을 높이는 것이 중요합니다.')

---

# Part 4: YOLOv8 기반 결함 탐지 실습 (30분) 🆕

## 4.1 YOLO란?

**YOLO (You Only Look Once)**는 실시간 객체 탐지 알고리즘입니다.

### 분류 vs 객체 탐지

| 구분 | 분류 (Classification) | 객체 탐지 (Object Detection) |
|------|----------------------|-----------------------------|
| **출력** | 클래스 라벨 (OK/NG) | 바운딩 박스 + 클래스 |
| **장점** | 단순, 빠름 | 결함 위치 특정 가능 |
| **단점** | 위치 정보 없음 | 라벨링 비용 높음 |
| **적용** | 전체 양/불 판정 | 결함 종류별 위치 파악 |

### YOLOv8 특징
- Ultralytics에서 개발한 최신 YOLO 버전
- 사용하기 쉬운 Python API
- 탐지, 세그멘테이션, 분류, 포즈 추정 지원
- COCO 데이터셋 기준 최고 수준 성능

In [ ]:
# Step 8: YOLOv8 설치
!pip install -q ultralytics

from ultralytics import YOLO
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

print('✅ YOLOv8 설치 완료!')

## 4.2 결함 탐지용 샘플 데이터 생성

실제 제조 현장에서는 다양한 결함 유형이 있습니다:
- **Scratch**: 스크래치
- **Dent**: 찌그러짐
- **Stain**: 얼룩
- **Crack**: 균열

In [ ]:
# Step 9: 결함 탐지용 시뮬레이션 데이터 생성
import os
import yaml

# 디렉토리 생성
!mkdir -p yolo_data/images/train yolo_data/images/val
!mkdir -p yolo_data/labels/train yolo_data/labels/val

def create_defect_image(img_size=640, defects=None):
    '''
    결함이 있는 제품 이미지 생성
    Returns: (이미지, 라벨 리스트)
    라벨 형식: [class_id, x_center, y_center, width, height] (정규화)
    '''
    # 배경 이미지 (금속 표면 시뮬레이션)
    img = np.ones((img_size, img_size, 3), dtype=np.uint8) * 180
    img = img + np.random.randint(-15, 15, img.shape).astype(np.int16)
    img = np.clip(img, 0, 255).astype(np.uint8)

    labels = []

    if defects is None:
        defects = random.sample(['scratch', 'dent', 'stain', 'crack'],
                                k=random.randint(1, 3))

    class_map = {'scratch': 0, 'dent': 1, 'stain': 2, 'crack': 3}

    for defect in defects:
        # 랜덤 위치 및 크기
        x = random.randint(50, img_size - 100)
        y = random.randint(50, img_size - 100)

        if defect == 'scratch':
            # 스크래치: 긴 직선
            w, h = random.randint(80, 150), random.randint(3, 8)
            cv2.rectangle(img, (x, y), (x+w, y+h), (40, 40, 40), -1)

        elif defect == 'dent':
            # 찌그러짐: 원형
            r = random.randint(15, 35)
            w, h = r*2, r*2
            cv2.circle(img, (x+r, y+r), r, (100, 100, 100), -1)
            cv2.circle(img, (x+r, y+r), r-3, (140, 140, 140), -1)

        elif defect == 'stain':
            # 얼룩: 불규칙한 원
            w, h = random.randint(40, 80), random.randint(40, 80)
            cv2.ellipse(img, (x+w//2, y+h//2), (w//2, h//2), 0, 0, 360, (80, 70, 60), -1)

        elif defect == 'crack':
            # 균열: 지그재그 선
            w, h = random.randint(60, 100), random.randint(40, 70)
            pts = np.array([[x, y], [x+w//3, y+h//2], [x+2*w//3, y+h//4], [x+w, y+h]])
            cv2.polylines(img, [pts], False, (30, 30, 30), 2)

        # YOLO 형식 라벨 (정규화된 좌표)
        x_center = (x + w/2) / img_size
        y_center = (y + h/2) / img_size
        w_norm = w / img_size
        h_norm = h / img_size
        labels.append([class_map[defect], x_center, y_center, w_norm, h_norm])

    return img, labels

# 학습/검증 데이터 생성
np.random.seed(42)
random.seed(42)

for split, n_images in [('train', 100), ('val', 20)]:
    for i in range(n_images):
        img, labels = create_defect_image()

        # 이미지 저장
        cv2.imwrite(f'yolo_data/images/{split}/{i:04d}.jpg', img)

        # 라벨 저장
        with open(f'yolo_data/labels/{split}/{i:04d}.txt', 'w') as f:
            for label in labels:
                f.write(' '.join(map(str, label)) + '\n')

print('✅ YOLO 학습 데이터 생성 완료!')
print(f'   - 학습 이미지: {len(os.listdir("yolo_data/images/train"))}개')
print(f'   - 검증 이미지: {len(os.listdir("yolo_data/images/val"))}개')

In [ ]:
# YOLO 데이터셋 설정 파일 생성
data_yaml = {
    'path': '/content/yolo_data',  # Colab 경로
    'train': 'images/train',
    'val': 'images/val',
    'names': {
        0: 'scratch',
        1: 'dent',
        2: 'stain',
        3: 'crack'
    }
}

with open('yolo_data/data.yaml', 'w') as f:
    yaml.dump(data_yaml, f)

print('✅ 데이터셋 설정 파일 생성: yolo_data/data.yaml')
print('\n클래스 목록:')
for k, v in data_yaml['names'].items():
    print(f'   {k}: {v}')

In [ ]:
# 생성된 데이터 시각화
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

class_names = ['scratch', 'dent', 'stain', 'crack']
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0)]

for idx, ax in enumerate(axes.flatten()):
    img_path = f'yolo_data/images/train/{idx:04d}.jpg'
    label_path = f'yolo_data/labels/train/{idx:04d}.txt'

    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    # 라벨 읽기 및 바운딩 박스 그리기
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            cls_id = int(parts[0])
            x_center, y_center, bw, bh = map(float, parts[1:])

            # 정규화 좌표를 픽셀 좌표로 변환
            x1 = int((x_center - bw/2) * w)
            y1 = int((y_center - bh/2) * h)
            x2 = int((x_center + bw/2) * w)
            y2 = int((y_center + bh/2) * h)

            cv2.rectangle(img, (x1, y1), (x2, y2), colors[cls_id], 2)
            cv2.putText(img, class_names[cls_id], (x1, y1-5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[cls_id], 2)

    ax.imshow(img)
    ax.set_title(f'Sample {idx+1}')
    ax.axis('off')

plt.suptitle('결함 탐지 학습 데이터 샘플 (바운딩 박스 포함)', fontsize=14)
plt.tight_layout()
plt.show()

## 4.3 YOLOv8 모델 학습

In [ ]:
# Step 10: YOLOv8 모델 학습
from ultralytics import YOLO

# 사전학습된 YOLOv8n 모델 로드 (nano - 가장 작은 모델)
yolo_model = YOLO('yolov8n.pt')

print('🚀 YOLOv8 학습 시작!\n')

# 학습 실행
results = yolo_model.train(
    data='yolo_data/data.yaml',
    epochs=30,           # 에포크 수
    imgsz=640,           # 이미지 크기
    batch=16,            # 배치 크기
    patience=10,         # 조기 종료 patience
    device=0 if torch.cuda.is_available() else 'cpu',
    verbose=True,
    plots=True           # 학습 그래프 생성
)

print('\n✅ YOLOv8 학습 완료!')

In [ ]:
# 학습 결과 시각화 (자동 생성된 그래프)
from IPython.display import Image as IPImage, display
import glob

# 최신 학습 결과 폴더 찾기
train_dirs = sorted(glob.glob('runs/detect/train*'))
if train_dirs:
    latest_dir = train_dirs[-1]
    print(f'📊 학습 결과 폴더: {latest_dir}')

    # 결과 이미지 표시
    result_images = ['results.png', 'confusion_matrix.png', 'F1_curve.png', 'PR_curve.png']

    for img_name in result_images:
        img_path = f'{latest_dir}/{img_name}'
        if os.path.exists(img_path):
            print(f'\n📈 {img_name}:')
            display(IPImage(filename=img_path, width=800))

## 4.4 YOLOv8 모델 검증 및 추론

In [ ]:
# Step 11: 학습된 모델로 검증 데이터 평가
# 최적 가중치 로드
best_model_path = f'{latest_dir}/weights/best.pt'
trained_model = YOLO(best_model_path)

# 검증 실행
val_results = trained_model.val(data='yolo_data/data.yaml')

print('\n📊 검증 결과:')
print(f'   mAP50: {val_results.box.map50:.4f}')
print(f'   mAP50-95: {val_results.box.map:.4f}')

In [ ]:
# Step 12: 새로운 이미지에 대한 추론
# 테스트 이미지 생성
test_img, test_labels = create_defect_image()
cv2.imwrite('test_defect.jpg', test_img)

# 추론 실행
results = trained_model.predict(
    source='test_defect.jpg',
    conf=0.25,           # 신뢰도 임계값
    iou=0.45,            # NMS IOU 임계값
    save=True,           # 결과 이미지 저장
    save_txt=True        # 라벨 저장
)

print('✅ 추론 완료!')

In [ ]:
# 추론 결과 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 원본 이미지
original = cv2.imread('test_defect.jpg')
original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
axes[0].imshow(original)
axes[0].set_title('원본 이미지')
axes[0].axis('off')

# 탐지 결과
result_img = results[0].plot()  # 바운딩 박스가 그려진 이미지
result_img = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
axes[1].imshow(result_img)
axes[1].set_title('YOLOv8 탐지 결과')
axes[1].axis('off')

plt.tight_layout()
plt.show()

# 탐지 결과 상세 정보
print('\n📋 탐지된 결함:')
class_names = ['scratch', 'dent', 'stain', 'crack']
for box in results[0].boxes:
    cls_id = int(box.cls[0])
    conf = float(box.conf[0])
    x1, y1, x2, y2 = box.xyxy[0].tolist()
    print(f'   - {class_names[cls_id]}: 신뢰도 {conf:.2%}, 위치 ({x1:.0f}, {y1:.0f}) - ({x2:.0f}, {y2:.0f})')

## 4.5 실시간 추론 시뮬레이션

In [ ]:
# Step 13: 배치 추론 및 성능 측정
import time

# 여러 테스트 이미지 생성
!mkdir -p test_images
n_test = 10

for i in range(n_test):
    img, _ = create_defect_image()
    cv2.imwrite(f'test_images/test_{i:03d}.jpg', img)

# 추론 시간 측정
start_time = time.time()
batch_results = trained_model.predict(
    source='test_images/',
    conf=0.25,
    save=False,
    verbose=False
)
total_time = time.time() - start_time

print('⏱️ 추론 성능:')
print(f'   총 이미지: {n_test}개')
print(f'   총 소요 시간: {total_time:.2f}초')
print(f'   이미지당 평균: {total_time/n_test*1000:.1f}ms')
print(f'   FPS: {n_test/total_time:.1f}')

# 결과 요약
print('\n📊 탐지 결과 요약:')
defect_counts = {'scratch': 0, 'dent': 0, 'stain': 0, 'crack': 0}
for result in batch_results:
    for box in result.boxes:
        cls_id = int(box.cls[0])
        defect_counts[class_names[cls_id]] += 1

for defect, count in defect_counts.items():
    print(f'   {defect}: {count}개')

## 4.6 모델 내보내기 (ONNX)

In [ ]:
# Step 14: ONNX 형식으로 내보내기
# Edge 디바이스나 다른 플랫폼에서 사용할 수 있도록 변환

trained_model.export(
    format='onnx',
    imgsz=640,
    simplify=True,
    dynamic=False
)

print('✅ ONNX 모델 내보내기 완료!')
print(f'   저장 위치: {best_model_path.replace(".pt", ".onnx")}')

---

# 📌 핵심 정리

## 분류 모델 (ResNet)
1. **전이학습:** 사전학습된 모델을 활용해 적은 데이터로도 높은 성능을 달성할 수 있습니다.
2. **혼동행렬:** TP, TN, FP, FN을 통해 모델 성능을 다각도로 평가합니다.
3. **임계값 조정:** 비즈니스 요구에 맞게 검출 민감도를 조절합니다.

## 객체 탐지 모델 (YOLO)
4. **YOLOv8:** 결함의 위치와 종류를 동시에 탐지할 수 있습니다.
5. **mAP:** 객체 탐지 모델의 성능을 평가하는 핵심 지표입니다.
6. **실시간 처리:** YOLO는 빠른 추론 속도로 라인 적용에 적합합니다.

---

# 🚀 도전 과제

1. 데이터 증강(Data Augmentation)을 추가하여 성능을 개선해 보세요.
2. YOLOv8s, YOLOv8m 등 더 큰 모델로 학습하여 성능을 비교해 보세요.
3. 실제 MVTec AD 데이터셋을 다운로드하여 테스트해 보세요.
4. YOLOv8-seg를 사용하여 세그멘테이션 모델을 학습해 보세요.

In [ ]:
# [도전 과제] 데이터 증강 예시
# YOLOv8은 자동으로 데이터 증강을 적용하지만, 추가 설정이 가능합니다.

# 커스텀 데이터 증강 설정
augmentation_config = {
    'hsv_h': 0.015,      # 색상 변화
    'hsv_s': 0.7,        # 채도 변화
    'hsv_v': 0.4,        # 명도 변화
    'degrees': 10,       # 회전
    'translate': 0.1,    # 이동
    'scale': 0.5,        # 스케일
    'shear': 5,          # 기울임
    'flipud': 0.5,       # 상하 반전
    'fliplr': 0.5,       # 좌우 반전
    'mosaic': 1.0,       # 모자이크 증강
    'mixup': 0.1         # 믹스업 증강
}

print('💡 데이터 증강 설정 예시:')
for k, v in augmentation_config.items():
    print(f'   {k}: {v}')
print('\n위 설정을 yolo_model.train()에 전달하여 사용할 수 있습니다.')

In [ ]:
# 모델 저장
torch.save(model.state_dict(), 'classification_model.pth')
print('✅ 분류 모델 저장: classification_model.pth')
print(f'✅ YOLO 모델 저장: {best_model_path}')
print('\n🎉 실습 1 완료! 다음 실습으로 넘어가세요.')

---

# 🔍 분류 vs YOLO 비교 요약

| 항목 | 분류 (ResNet) | 객체 탐지 (YOLO) |
|------|---------------|------------------|
| **출력** | OK/NG 라벨 | 바운딩 박스 + 클래스 |
| **장점** | 단순, 빠른 학습 | 결함 위치/종류 특정 |
| **단점** | 위치 정보 없음 | 라벨링 비용 높음 |
| **학습 데이터** | 이미지 + 라벨 | 이미지 + 바운딩 박스 |
| **추론 속도** | ~5ms | ~15ms |
| **적용 사례** | 전체 양/불 판정 | 결함 상세 분석 |

**💡 실무 팁:** 두 모델을 조합하여 사용하는 것이 효과적입니다!
1. **1차 검사**: 분류 모델로 빠르게 OK/NG 판정
2. **2차 검사**: NG 판정된 제품만 YOLO로 결함 상세 분석

---

**➡️ 다음 실습: 예지보전 & 공정 최적화**